In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/jacobgil/pytorch-grad-cam.git

Cloning into 'pytorch-grad-cam'...
remote: Enumerating objects: 1115, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 1115 (delta 4), reused 6 (delta 2), pack-reused 1097
Receiving objects: 100% (1115/1115), 109.60 MiB | 26.30 MiB/s, done.
Resolving deltas: 100% (615/615), done.


In [3]:
import sys
sys.path.append('/content/drive/pytorch-grad-cam')
sys.path.append('/content/drive/pytorch-grad-cam/pytorch-grad-cam')

In [4]:
%cd pytorch-grad-cam

/content/pytorch-grad-cam


In [5]:
!pip install -r requirements.txt
!pip install pytorch-lightning
!pip install torch-lr-finder
!pip install grad-cam
!pip install lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.7/727.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 25.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for grad-cam: filename=grad_cam-1.4.8-py3-none-any.whl size=38243 sha256=2471636deabbcc013c899c24a0500717543d4ff6202881509c376966aeb250b9
  Stored in directory: /root/.cache/pip/wheels/f8/04/36/94ff3c8a4215826a21946b34c01180817e606989fdf53f7cd6
Successfully built grad-cam
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.3/71.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [6]:
sys.path.append('/content/drive/MyDrive/lightning-wrapper')

In [7]:
from models.model import *

In [8]:
model = LitModel()

In [10]:
%cd /content/drive/MyDrive/lightning-wrapper

/content/drive/MyDrive/lightning-wrapper


In [11]:
%ls

data/    models/     README.md        saved_model_lightning.pth
LICENSE  notebooks/  requirement.txt  utils/


In [12]:
model.load_state_dict(torch.load('saved_model_lightning.pth'))
model.eval()

LitModel(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential()
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2

In [13]:
from data.cifar10_datamodule import *

In [14]:
cifar10_dm = CIFAR10DataModule()

In [15]:
# Prepare and setup data
cifar10_dm.prepare_data()
cifar10_dm.setup()

100%|██████████| 170498071/170498071 [00:03<00:00, 43928599.34it/s]


Extracting ./cifar-10-python.tar.gz to .
Files already downloaded and verified


In [17]:
# Get test DataLoader
test_loader = cifar10_dm.val_dataloader()

Files already downloaded and verified


In [18]:
from utils.gradcam import *

In [ ]:
target_layer = -2
preprocess_args = {
    'mean': [0.4914, 0.4822, 0.4465],
    'std': [0.2470, 0.2435, 0.2616]
}
